<h1>Graph based Methods for Text Summarization</h1>
<br>Darragh Clabby
<br>HDAIML_SEP
<br>20142935
<br>National College of Ireland
<br>Dublin, Ireland
<br>x20142935@student.ncirl.ie
<br>
<h2>Abstract</h2>
This project explores the application of the
TextRank algorithm to the task of automatic text
summarization. The TextRank algorithm ranks the importance
of sentences in a text by representing the text as a graph, with
vertices representing sentences and edges representing
similarity between sentences. The texts to be summarized were
news articles taken from the Daily Mail data set. Each text
contained in the data set is accompanied by a human generated
summary which may be used as reference summary against
which automatically generated summaries may be assessed.
Summarization was implemented according to five methods,
three of which employed the TextRank algorithm. The aim of
this work was to assess how the basic TextRank method
compared to the simpler TF-IDF method; and to assess
whether sentence embedding and unsupervised classification
could be employed to further improve performance. The
results suggest that the basic TextRank performed best (highest
accuracy, lowest computational demand). The methods
employing sentence embedding and unsupervised classification
did not deliver further performance improvements.

<br>
<h2>Import Libraries</h2>

In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import os
import time

<br>
<h2>Import & Clean Data</h2>
<!--<h3>Import BBC Data</h3>-->

In [2]:
def load_DailyMail_Data(fileList, folderName=""):
    summary = []
    text = []
    nLinesSummary = []
    for iFile in fileList:
        
        tmpFile = open(folderName + "/" + iFile, 'r')
        rawText = tmpFile.readlines()
        tmpFile.close()
        
        strippedText = ""
        for t in rawText:
            strippedText += t
        
        strippedText = strippedText.split("@highlight") #separate highlights from the main article by splitting on "@highlight"
        nLinesSummary.append(len(strippedText) - 1)
        textTmp = strippedText[0] # article is the first element in the split text
        summaryTmp = "" # initialize string for the summary
        for h in strippedText[1:]: # iterate through the highlights (all elements of the split text other than the first)
            summaryTmp += h + ". " # append highlights to the summary string, & add full stop and space between highlights
        
        summary.append(summaryTmp)
        text.append(textTmp)
    
    df = pd.DataFrame({"Source": fileList, "Summary":summary, "Text": text, "nLinesSummary": nLinesSummary})
    return df

def cleanCorpus(textSeries, removeStopWords=True):
    cleanText = []
    rawText = []
    for text in textSeries:
        cleanList, rawList = cleanDocument(text, removeStopWords)
        cleanText.append(cleanList)
        rawText.append(rawList)
    return cleanText, rawText

def cleanDocument(rawText, removeStopWords=True):
    """
    will return:
    - clean list: [ [sentence], [[word], [word]] ]
    - raw list: [[sentence], [sentence]]
    """
    
    rawText = rawText.replace("\n", " .").replace("\xa0", " .") # replace end of line tage with " ."
    rawText = nltk.tokenize.sent_tokenize(rawText)
        
    tr = str.maketrans("", "", string.punctuation)
    remove_digits = str.maketrans('', '', string.digits)
    stop_words = nltk.corpus.stopwords.words("english")
    
    cleanList = []
    rawList = []
    for sentence in rawText:
        sentence = sentence.lstrip(".")
        cleanSentence = sentence.translate(tr) # remove punctuation from string
        cleanSentence = cleanSentence.translate(remove_digits) # remove numbers
        cleanSentence = cleanSentence.lower() # convert all characters to lower
        if removeStopWords:
            cleanSentence = [s for s in nltk.tokenize.word_tokenize(cleanSentence) if s not in stop_words] # remove stop words & split sentence into words
        else:            
            cleanSentence = [s for s in nltk.tokenize.word_tokenize(cleanSentence)] 
        if len(cleanSentence):
            cleanList.append(cleanSentence)
            rawList.append(sentence)
    
    return cleanList, rawList

def printLines(rawSentences, iFile):
    for i, s in enumerate(rawSentences[iFile]):
        print("\nSentence " + str(i) + ": " + str(s) + "\n")

<br>
<h2>TF-IDF</h2>
<h3>Calculate TF-IDF Coefficients</h3>

In [3]:
def calcTFIDFcoeffs(wordTokens):
    termCounts = pd.DataFrame()
    nDocs = len(wordTokens)
    for iDoc in range(0, nDocs):
        for sentence in wordTokens[iDoc]:
            for word in sentence:
                try:
                    termCounts[word][iDoc] += 1
                except:
                    termCounts[word] = pd.Series(np.zeros(nDocs).astype(int))
                    termCounts[word][iDoc] += 1

    nTerms = len(termCounts.columns)

    C_i = termCounts.sum(axis=1)
    B_j = termCounts.astype(bool).sum(axis=0)

    TF =  np.array(termCounts)/ np.transpose(np.tile(C_i, (nTerms, 1)))
    IDF = np.log(nDocs/np.array(B_j))
    TFIDF = TF*np.tile(IDF, (nDocs, 1))

    colNames = [c for c in termCounts]
    TF = pd.DataFrame(TF, columns = colNames)
    IDF = pd.Series(IDF, index = colNames)
    TFIDF = pd.DataFrame(TFIDF, columns = colNames)
    return TFIDF, TF, IDF

<h3>Calculate Sentence Scores</h3>

In [4]:
def calcScoresTFIDF(wordTokens, TFIDF, sentenceTokens, docIndices=[]):
    
    if len(docIndices) == 0:
        docIndices = [x for x in range(0,len(cleanWords))]
    
    scores = []
    for iDoc in docIndices:
        doc = wordTokens[iDoc]    
        docScores = []
        for sentence in doc:
            sentenceScore = 0
            
            #if len(sentence) == 0:
            #    print(iDoc)
            
            #for word in sentence:
            #    sentenceScore += TFIDF[word][iDoc]
            #sentenceScore /= len(sentence)
            if len(sentence) > 0:
                for word in sentence:
                    sentenceScore += TFIDF[word][iDoc]
                sentenceScore /= len(sentence)
            docScores.append(sentenceScore)
        dfTmp = pd.DataFrame({"rawSentence": sentenceTokens[iDoc], "sentenceScore": docScores})
        scores.append(dfTmp)
        
        #scores.append(docScores)
    
    return scores   

In [5]:
def generateSummary(sentenceScores, nSentences=3):
    summary = []
    for doc in sentenceScores:
        #avgScore = doc['sentenceScore'].mean()
        #summaryData = doc[doc['sentenceScore'] > 1.5*avgScore]
        #n = 3#len(doc)//5
        summaryData = doc.sort_values(by=['sentenceScore'], ascending = False)
        summaryData = summaryData.head(nSentences)
        summaryData = summaryData.sort_index()
        
        summaryTmp = ""
        for sentence in summaryData['rawSentence']:
            summaryTmp += sentence
        summary.append(summaryTmp)
        #summary.append(summaryData)
    return summary

<br>
<h2>TextRank</h2>

In [6]:
def sentenceSimilarity(wordTokens, docIndices=[]):#, sentenceTokens, docIndices=[]):
    
    if len(docIndices) == 0:
        docIndices = [x for x in range(0,len(cleanWords))]
        
    simMatrices = []
    for iDoc in docIndices:
        doc = wordTokens[iDoc]    
        nSentences = len(doc)
        denom = np.ones((nSentences, nSentences))
        docSim = np.zeros((nSentences, nSentences))
        for i, sentence_i in enumerate(doc):
            #n_i = len(set(sentence_i)) # number of unique words in ith sentence
            n_i = len(sentence_i) # number of words in ith sentence
            for j, sentence_j in enumerate(doc):                
                #n_j = len(set(sentence_j)) # number of unique words in jth sentence
                n_j = len(sentence_j) # number of words in jth sentence
                #if i > j: # only need to calculate lower triangular matrix since edges undirected and matrix is symmetrical (similarity i to j = similarity j to i)
                if i != j:
                    #if n_i > 0 and n_j > 0:
                    denom[i, j] = np.log(n_i) + np.log(n_j)
                    if denom[i, j] > 0:
                        docSim[i, j] = len(set(sentence_i).intersection(set(sentence_j)))/denom[i, j]
        simMatrices.append(docSim)
    return simMatrices   

In [7]:
def calcScoresTextRank(similarityMatrix, rawSentences, d=0.85, initialScores=1, tol = 0.0001, docIndices=[]):
    
    textRankScores = []
    for iDoc, simMat in enumerate(similarityMatrix):
        simMat = np.abs(simMat) #NOTE: cosine similarity has a range -1 to 1 whereas text rank expects similarity in the range 0 to 1
        nSentences = np.shape(simMat)[0]
        sentenceScores = np.ones((nSentences))*initialScores
        prev = np.zeros((nSentences))

        #N = 30
        #error = []
        #for n in range(0,N):
        error = 1
        while error > tol:
            for i, score_i in enumerate(sentenceScores):
                tmp = 0
                for j, score_j in enumerate(sentenceScores):
                    w_ji = simMat[j, i]
                    w_jk = np.sum(simMat[j])
                    #if w_jk > 0:
                    if w_jk != 0:
                        tmp += w_ji*score_j/w_jk

                sentenceScores[i] = (1-d) + d*tmp
            #error.append(np.sum(np.abs(sentenceScores[i] - prev)))
            error = np.sum(np.abs(sentenceScores[i] - prev))
            prev = sentenceScores[i]
        #if len(rawSentences) != len(sentenceScores):
        #    print("sentence length: " + str(len(rawSentences)) + "; \nscore length: " + str(len(sentenceScores)) + "\n")
        #    return []
        df = pd.DataFrame({"rawSentence": rawSentences[iDoc], "sentenceScore": sentenceScores})
        #return df
    
        textRankScores.append(df)
    return textRankScores



<br>
<h2>Sentence Embeddings</h2>
<h3>Import Sentence Transformer</h3>

In [8]:
# from: https://www.analyticsvidhya.com/blog/2020/08/top-4-sentence-embedding-techniques-using-python/
#!pip install sentence-transformers
from sentence_transformers import SentenceTransformer
#sbert_model = SentenceTransformer('bert-base-nli-mean-tokens') # why was this model chosen? see https://www.sbert.net/docs/pretrained_models.html
sbert_model = SentenceTransformer('stsb-roberta-base') # optimized for Semantic Textual Similarity (STS). 

/home/dclabby/anaconda3/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


<h3>Transform Sentences & Calculate Similarity</h3>

In [9]:
def cosineSimilarity(u, v):
    return (np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))).item()


def generateSentenceVecs(sentenceTokens, sbert_model, docIndices=[]):
    if len(docIndices) == 0:
        docIndices = [x for x in range(0,len(cleanWords))]
        
    sentenceVectors = []
    for iDoc in docIndices:
        doc = sentenceTokens[iDoc]
        sentenceVectors.append(sbert_model.encode(doc))
    return sentenceVectors


#def sentenceEmbeddingSimilarity(sentenceTokens, sbert_model, docIndices=[]):
def sentenceEmbeddingSimilarity(docVecs, docIndices=[]):
    
    if len(docIndices) == 0:
        docIndices = [x for x in range(0,len(cleanWords))]
        
    simMatrices = []
    for iDoc in docIndices:
    #    doc = sentenceTokens[iDoc]
    #    sentenceVectors = sbert_model.encode(doc)
        sentenceVectors = docVecs[iDoc]
                
        #nSentences = len(doc)
        nSentences = len(sentenceVectors)
        docSim = np.zeros((nSentences, nSentences))
        for i, vec_i in enumerate(sentenceVectors):
            for j, vec_j in enumerate(sentenceVectors):
                if i != j:
                    docSim[i, j] = cosineSimilarity(vec_i, vec_j)
        
        simMatrices.append(docSim)
    return simMatrices   


<br>
<h2>K-Nearest Neighbours</h2>
<h3>KNN Algorithm</h3>

In [10]:
def knn(data, k):
    """
    data is an array with:
    - dimension 0 (rows) containing sample points;
    - dimension 1 (columns) containing dimensions
    - example: data of size n x d contains n data points, each of dimension d
    - for a given document, this will take sentence vectors where n is the number of sentences & d is the size of the vector
    """
    nSamples = np.shape(data)[0]
    D = np.shape(data)[1]
    
    #assign random starting positions
    i_z = np.random.choice(nSamples, k, replace=False)
    z_k = data[i_z, :]
    z_initial = np.array(z_k, copy=True)
    
    # iteratively assign data to nearest z
    continueLoop = True # initialize variable to terminate while loop
    prevClusterNos = np.zeros(nSamples) # initialize array of zeros to store previous cluster assignments
    n = 0 # initialize loop counter
    maxIts = 2*nSamples # define maximum iterations to prevent infinite loop
    while continueLoop and n < maxIts:    
        n += 1 # increment the loop counter
        cost = np.zeros([nSamples, k]) # initialize the cost matrix
        for i, z in enumerate(z_k): # iterate through each cluster...
            cost[:, i] = np.linalg.norm(data - z_k[i],axis=1) # calculate the distance between each data point and the ith cluster centre
        clusterNos = np.argmin(cost,axis=1) # identify the closest cluster to each data point
        clusterData = []
        for i_k in range(0,k): # iterate through each cluster
            clusterData.append(data[clusterNos == i_k, :]) # extract the data for the ith cluster
            z_k[i_k, :] =  np.mean(clusterData[-1], axis=0) # update the centre of the cluster based on the average of its data
        continueLoop = not np.all(clusterNos == prevClusterNos) # continue the loop if the present and previous cluster assignments differ
        prevClusterNos = clusterNos # update previous cluster numbers (for the next iteration) based on this iteration's cluster numbers
        
    return clusterData, clusterNos



<br>
<h3>Clustering</h3>

In [11]:
def calcClusterCentres(docVecs, rawSentences, k):

    clusterCentreScores = []
    for sentenceVecs, sentencesRaw in zip(docVecs, rawSentences):
        sentenceClusters, clusterNos = knn(sentenceVecs, k)
        clusterDicts = []
        for i_c, cluster in enumerate(sentenceClusters):
            clusterMean = np.mean(cluster,axis=0)
            clusterScores = []
            for u in cluster:
                clusterScores.append(cosineSimilarity(u, clusterMean)) # cosine similarity is not sensitive to sentence size
                #clusterScores.append(np.linalg.norm(u - clusterMean)) 

            clusterSentences = []
            for s, c in zip(sentencesRaw, clusterNos):
                if c == i_c:
                    clusterSentences.append(s)

            clusterDicts.append(pd.DataFrame({"rawSentence": clusterSentences, "sentenceScore": clusterScores}))
        clusterCentreScores.append(clusterDicts)
    return clusterCentreScores

In [12]:
def generateClusterSummaries(clusterCentreScores, nSentences=3):
    docSummary = []
    for docCluster in clusterCentreScores:
        nClusters = len(docCluster)
        sentencesPerCluster = [len(s) for s in docCluster]
        sentencesPerCluster = np.round((sentencesPerCluster/np.sum(sentencesPerCluster))*nSentences).astype('int')
        #summary = []
        summaryTmp = ""
        for clusterData, nC in zip(docCluster, sentencesPerCluster):
            if nC > 0:
                                
                summaryData = clusterData.sort_values(by=['sentenceScore'], ascending = False)
                summaryData = summaryData.head(nC)
                summaryData = summaryData.sort_index()

                #summaryTmp = ""
                for sentence in summaryData['rawSentence']:
                    summaryTmp += sentence
            #summary.append(summaryTmp)
                
        docSummary.append(summaryTmp)
        #docSummary.append(summary)
    return docSummary

<br>
<h3>TextRank Clustering</h3>

In [13]:
def calcClusterRank(textRankScores, clusterCentres):
    clusterRank = []
    for tr, cc in zip(textRankScores, clusterCentres):
        clusterRankTmp = []
        for c in cc:
            clusterRankTmp.append(pd.merge(tr, c, on="rawSentence").rename(columns={"sentenceScore_x": "sentenceScore"}))
        clusterRank.append(clusterRankTmp)
    return clusterRank

<br>
<h2>Evaluate Summaries</h2>

In [14]:
def calcRogue(candidate, reference, removeStopWords=True):
    rogueScores = []
    for c, r in zip(candidate, reference):
        candidateClean, candidateRaw = cleanDocument(c, removeStopWords)
        referenceClean, referenceRaw = cleanDocument(r, removeStopWords)

        candidateSet = set(sum(candidateClean, []))
        referenceSet = set(sum(referenceClean, []))
        rogueScores.append(len(candidateSet.intersection(referenceSet))/len(referenceSet))
    return rogueScores



<br>
<h2>Implement Tests</h2>
<h3>0. Load Data</h3>

In [21]:
## 0. Load Data
#folderName = "/home/dclabby/Documents/Springboard/HDAIML_SEP/Semester02/ArtIntel/Project/TextSummCode/data"
folderName = "./data"
folderContents = os.listdir(folderName)
nFiles = 10
files = folderContents[0:nFiles]
dmData = load_DailyMail_Data(files, folderName)
cleanWords, rawSentences = cleanCorpus(dmData["Text"])
ref = dmData["Summary"]
N = np.round(dmData["nLinesSummary"].mean()).astype('int').item()
#r = pd.DataFrame({})

<br>
<h3>1. Main</h3>

In [22]:
rogueScore = pd.DataFrame()
execTime = []
wordCountRatio = []
rLength = np.array([len(d.split(" ")) for d in ref])

## 1. TF-IDF
print("Calculating TF-IDF...")
t1 = time.time()
tfidf, tf, idf = calcTFIDFcoeffs(cleanWords)
sentenceScores = calcScoresTFIDF(cleanWords, tfidf, rawSentences)
tfIdfSummaries = generateSummary(sentenceScores, N)

rogueScore["TF-IDF"] = calcRogue(tfIdfSummaries, ref)
execTime.append(time.time() - t1)
#wordCountRatio.append(np.mean(rLength/np.array([len(d.split(" ")) for d in tfIdfSummaries])))
wordCountRatio.append(rLength/np.array([len(d.split(" ")) for d in tfIdfSummaries]))

## 2. TextRank
print("Calculating TextRank...")
t1 = time.time()
trSimilarity = sentenceSimilarity(cleanWords)#, sentenceTokens)
textRankScores = calcScoresTextRank(trSimilarity, rawSentences)
textRankSummaries = generateSummary(textRankScores, N)

rogueScore["TextRank"] = calcRogue(textRankSummaries, ref)
execTime.append(time.time() - t1)
#wordCountRatio.append(np.mean(rLength/np.array([len(d.split(" ")) for d in textRankSummaries])))
wordCountRatio.append(rLength/np.array([len(d.split(" ")) for d in textRankSummaries]))

# Generate Sentence Embeddings Separately
print("Generating Sentence Embeddings...")
t1 = time.time()
docVecs =  generateSentenceVecs(rawSentences, sbert_model)
tEmbedding = time.time() - t1
print("Sentence Embeddings completed in " + str(tEmbedding) + "s")

## 3. CosRank
print("Calculating CosRank...")
t1 = time.time()
#docVecs =  generateSentenceVecs(rawSentences, sbert_model)
cosSimMat = sentenceEmbeddingSimilarity(docVecs)
cosRankScores = calcScoresTextRank(cosSimMat, rawSentences) 
cosRankSummaries = generateSummary(cosRankScores, N)

rogueScore["CosRank"] = calcRogue(cosRankSummaries, ref)
execTime.append(time.time() - t1 + tEmbedding)
#wordCountRatio.append(np.mean(rLength/np.array([len(d.split(" ")) for d in cosRankSummaries])))
wordCountRatio.append(rLength/np.array([len(d.split(" ")) for d in cosRankSummaries]))

## 4. Cluster Center
print("Calculating Cluster Centres...")
t1 = time.time()
#docVecs =  generateSentenceVecs(rawSentences, sbert_model)
k = N
np.random.seed(0)
clusterCentres = calcClusterCentres(docVecs, rawSentences, k)
clusterCentreSummaries = generateClusterSummaries(clusterCentres, N)

rogueScore["ClusterCentre"] = calcRogue(clusterCentreSummaries, ref)
execTime.append(time.time() - t1 + tEmbedding)
#wordCountRatio.append(np.mean(rLength/np.array([len(d.split(" ")) for d in clusterCentreSummaries])))
wordCountRatio.append(rLength/np.array([len(d.split(" ")) for d in clusterCentreSummaries]))

## 5. Cluster Rank
print("Calculating ClusterRank...")
t1 = time.time()
#trSimilarity = sentenceSimilarity(cleanWords)#, sentenceTokens)
#textRankScores = calcScoresTextRank(trSimilarity, rawSentences)
#docVecs =  generateSentenceVecs(rawSentences, sbert_model)

np.random.seed(0)
clusterCentres = calcClusterCentres(docVecs, rawSentences, k)
#clusterRank = calcClusterRank(textRankScores, clusterCentres)
clusterRank = calcClusterRank(cosRankScores, clusterCentres)
clusterRankSummaries = generateClusterSummaries(clusterRank, N)

rogueScore["ClusterRank"] = calcRogue(clusterRankSummaries, ref)
execTime.append(time.time() - t1 + tEmbedding)
#wordCountRatio.append(np.mean(rLength/np.array([len(d.split(" ")) for d in clusterRankSummaries])))
wordCountRatio.append(rLength/np.array([len(d.split(" ")) for d in clusterRankSummaries]))



Calculating TF-IDF...
Calculating TextRank...
Generating Sentence Embeddings...
Sentence Embeddings completed in 19.47514319419861s
Calculating CosRank...
Calculating Cluster Centres...
Calculating ClusterRank...


<br><h1>Appendix</h1>
The remaining cells can be used to explore and export results

In [23]:
iFile=8
print("\nReference Summary:\n" + dmData["Summary"][iFile] + "\n")
print("\nTF-IDF:\n" + tfIdfSummaries[iFile] + "\n")
print("\nTextRank:\n" + textRankSummaries[iFile] + "\n")
print("\nCosRank:\n" + cosRankSummaries[iFile] + "\n")
print("\nClusterCenter:\n" + clusterCentreSummaries[iFile] + "\n")
print("\nClusterRank:\n" + clusterRankSummaries[iFile] + "\n")


Reference Summary:


Showbiz party held in honour of a children's charity and featured a marmot

. 

Russian celebrities happily posed with the creature as they arrived at event

. 

But riot erupted when host announced he was to 'kill, cook and serve' it

. 

Host said it would be 'entertaining to have a groundhog which didn't survive groundhog day'. The animal survived the ordeal and is now at zoo. 


TF-IDF:
He said he wanted to kill and cook it as he thought it would be 'entertaining to have a groundhog which didn't survive groundhog day'.He said he wanted to kill and cook it as he thought it would be 'entertaining to have a groundhog which didn't survive groundhog day' .The groundhog day special cake.'I just thought it would be entertaining to have a groundhog who didn't actually survive groundhog day.'


TextRank:
A riot erupted at a .Groundhog Day party in Russia after guests realised their host was planning to cook a marmot - and then serve it with cranberry sauce.A riot erupt

In [86]:
#Export Data
df2 = pd.DataFrame([execTime, np.mean(wordCountRatio,axis=1)], columns=['TF-IDF','TextRank','CosRank', 'ClusterCentre', 'ClusterRank'], index=['execTime','wordCountRatio'])
summaryDf = pd.concat([rogueScore, df2])
#summaryDf
#summaryDf.to_excel('tmp.xlsx')
#dmData.to_excel('tmp.xlsx')

In [8]:
#Move 1000 files
import os
from shutil import copyfile
sourceFolder = "/home/dclabby/Documents/DataScience/Data Sets/dailymail_stories/dailymail/stories/"
destFolder = "/home/dclabby/Documents/Springboard/HDAIML_SEP/Semester02/ArtIntel/Project/TextSummCode/data/"

folderContents = os.listdir(sourceFolder)
nFiles = 1000
files = folderContents[0:nFiles]

for iFile in files:
    copyfile(sourceFolder + iFile, destFolder + iFile)